In [4]:
import pandas as pd
import requests, json
import numpy as np
import folium
import re
import json
import datetime
import xml.etree.ElementTree as Xml

In [5]:
minlon, minlat, maxlon, maxlat = 77.7519486464,12.9833797192,77.7528560462,12.9842016832

minlat=str(minlat)
minlon=str(minlon)
maxlat=str(maxlat)
maxlon=str(maxlon)

In [6]:
hermap_filename = 'data_comparision/heremap.json'
sumo_filename = 'data_comparision/original_sumo.net.xml'

In [7]:
with open(hermap_filename) as f:
    heremap_json = json.load(f)

In [8]:
get_transform([1789.01,2940.24])

NameError: name 'get_transform' is not defined

In [9]:
def get_transform(list_point):
    x = list_point[0]
    y = list_point[1]
    loc = [77.735568,12.957348,77.757670,12.994528]
    points = [0.00,-0.00,2391.16,4129.88]
    l1 = loc[0]+((loc[2]-loc[0])/(points[2]-points[0]))*x
    l2 = loc[1]+((loc[3]-loc[1])/(points[3]-points[1]))*y
    return [l2,l1]

In [95]:
e = Xml.parse(sumo_filename).getroot()
osm=folium.Map(location=[12.96984,77.74992],zoom_start=20)

for i in e:
    if i.tag == 'edge':
        #print(i.attrib['shape'])
        
        for j in i:
            list_points = []
            if j.attrib['id'] in ["122516806_0", "122516806_1", "143215059_0" ,"143215059_1" ,"580921886#0_0" ,"580921886#0_1" ,"28043853_0" ,"28043853_1"]:
                print(j.attrib['id'])
                for ll in j.attrib['shape'].split():
                    list_points.append(get_transform([float(i) for i in ll.split(',')]))
                print(list_points)
            folium.PolyLine(locations=list_points,opacity=0.75,weight=4).add_to(osm)

122516806_0
[[12.983579297180547, 77.75379090101039], [12.983698402675138, 77.7528379258017], [12.983720009114066, 77.75268088362134], [12.983716498067741, 77.75266064098597], [12.983712266806783, 77.7526237605681]]
122516806_1
[[12.983608735953588, 77.75379478315965], [12.983727841448179, 77.7528419928152], [12.983750168101738, 77.75268032902859], [12.983745846813951, 77.75265611181185], [12.983742515821284, 77.75262644109972]]
143215059_0
[[12.994299331854679, 77.75763829578113], [12.99376223179366, 77.75752959560214], [12.993635654072275, 77.7575075967564], [12.993234404496015, 77.75747274984526], [12.993051469979758, 77.75738438473377], [12.992897884209711, 77.75728659154554], [12.99280515657598, 77.75720441938641], [12.991907048931205, 77.75652079138996], [12.991328176421591, 77.75605909292561], [12.990891906408903, 77.75571672433463], [12.990022877429853, 77.75509280749093], [12.989624778792605, 77.75488206224594], [12.988742335816053, 77.75453738285184], [12.988400323893188, 77.

In [93]:
        
osm

In [62]:
list_points

[[12.98389, 77.75212], [12.98381, 77.75237], [12.9838, 77.75245]]

In [13]:
def create_data_1(dictionary):
        
    RWS=dictionary['RWS']
    l=[]
    for rw in RWS:
        EBU_COUNTRY_CODE=rw['EBU_COUNTRY_CODE']
        EXTENDED_COUNTRY_CODE=rw['EXTENDED_COUNTRY_CODE']
        UNITS=rw['UNITS']
        for elem in rw['RW']:
            DE=elem['DE']
            LI=elem['LI']
            PBT=elem['PBT']
            mid=elem['mid']
            FIS=elem['FIS']
            for FI in FIS:
                elem_1=FI['FI']
                for elem_2 in elem_1:
                    LE=elem_2['TMC']['LE']
                    PC=elem_2['TMC']['PC']
                    QD=elem_2['TMC']['QD']
                    SHP=elem_2['SHP']
                    CF=elem_2['CF']
                    for elem_3 in CF:
                        FF=elem_3['FF']
                        CN=elem_3['CN']                    
                        JF=elem_3['JF']
                        SP=elem_3['SP']
                        TY=elem_3['TY']
                        l.append([EBU_COUNTRY_CODE,EXTENDED_COUNTRY_CODE,UNITS,DE,LI,PBT,mid,LE,PC,QD,SHP,FF,CN,JF,SP,TY])
    clms=['EBU_COUNTRY_CODE','EXTENDED_COUNTRY_CODE','UNITS','DE','LI','PBT','mid','LE','PC','QD','SHP','FF','CN','JF','SP','TY']                    
    df=pd.DataFrame(l,columns=clms)
    df['PBT']=datetime.datetime.now()
    #print(json.dumps(rws_list, indent=2))
#     if path_to_save:
#         if os.path.isfile(path_to_save):
#             df.to_csv(path_to_save,mode='a+',header=False,index=False)
#         else:
#             df.to_csv(path_to_save,mode='w+',index=False)
#     else:
    df['SHP'] = df['SHP'].astype(str)
    df['patch'] = df['PC'].map(str) + df['QD']
    return df  

In [14]:
def traffic_map_across_city_historical(traffic_data):
    
    traffic_hour= traffic_data
    df_jaipur=traffic_data
    df_ori=df_jaipur
    list_shp = df_jaipur['SHP'].unique()
    list_mean = []
    start_point=[]
    end_point=[]
    for s in list_shp:
        tmp = re.findall("[+-]?\d*\.\d+", s)
        #print(tmp)
        tmp=[float(i) for i in tmp]
        tmp2 = []
        it = iter(tmp)
        for z in it:
            tmp2.append([z, next(it)])
        a = np.array(tmp2)
        list_mean.append(list(np.mean(a, axis=0)))
        start_point.append(tmp2[0])
        end_point.append(tmp2[-1])
    df_t = pd.DataFrame()
    df_t['SHP'] = list_shp
    df_t['mean'] = list_mean
    df_final =df_jaipur.merge(df_t,on='SHP')
    df_final['latitude'] = np.array(df_final['mean'].values.tolist())[:,0]
    df_final['longitude'] = np.array(df_final['mean'].values.tolist())[:,1]
    hh=pd.pivot_table(df_final,index=['SHP','PC','DE'],values='JF',aggfunc='median').reset_index()
    da1=hh.sort_values('JF',ascending=False)
    #da2=da1[da1['JF']>5]
    da2=da1

    data_2=da2
    data_2=data_2
    import branca.colormap as cm

    colormap = cm.LinearColormap(['green','yellow','red'],vmin=0,vmax=10)
    colormap.caption="Jamfactor_levels"
#     lat_long=df[['latitude','longitude']].values[0].tolist()
    x=folium.Map(location=[12.9380131671,77.6319900345],zoom_start=10)
    # x

    for i in data_2.index.tolist():
        color_grad_val=data_2['JF'][i]
        tmp2=[]
        #tmp=re.findall("\d*\.\d+", data_2['SHP'][i])
        tmp = re.findall("[+-]?\d*\.\d+", data_2['SHP'][i])
        #print(tmp)
        tmp=[float(i) for i in tmp]

        it = iter(tmp)
        for z in it:
            tmp2.append([z, next(it)])
        y=folium.PolyLine(locations=tmp2,color=colormap.rgb_hex_str(color_grad_val),opacity=0.75,weight=7)
        x.add_children(y)
        
    
    df= df_final[['latitude','longitude','PC']].drop_duplicates()

    l = df[['latitude','longitude']].values.tolist()

    #map_osm = folium.Map(location=[38.946241233265095, -77.04115322627564],tiles='CartoDB Positron',
    #                     zoom_start=12)
    for i,k in zip(l,df['PC']):

        marker = folium.Marker(i,popup=str(k),icon=folium.Icon())

        x.add_child(marker)

    

    x
    display(x)

In [15]:
heremap_json
heremap_data = create_data_1(heremap_json)

In [16]:
traffic_map_across_city(heremap_data,'JF')

NameError: name 'traffic_map_across_city' is not defined

In [36]:
def traffic_map_across_city(x,df,label):
        #hh=pd.pivot_table(df,index=['SHP','PC','DE'],values='JF',aggfunc='median').reset_index()[:1000]
        #hh = df
        data_2=df#[df.QD == '-']
        if label == 'JF':
            colormap = folium.LinearColormap(['green','yellow','red'],vmin=0,vmax=10)
        else:
            colormap = folium.LinearColormap(['yellow','red'],vmin=df[label].min(),vmax=df[label].max())
        colormap.caption=label+"_levels"

        #x=folium.Map(location=[12.96984,77.74992],zoom_start=14)
        # x
        #x.add_child(colormap)
        for i in (data_2.index.tolist())[:6]:
            print(data_2['patch'][i])
            color_grad_val=data_2[label][i]
            tmp2=[]
            tmp=re.findall("\d*\.\d+", data_2['SHP'][i])
            tmp=[float(i) for i in tmp]

            it = iter(tmp)
            for z in it:
                tmp2.append([z, next(it)])

            y=folium.PolyLine(locations=tmp2,color=colormap.rgb_hex_str(color_grad_val),opacity=0.75,weight=4)
            x.add_child(y)
            

            print(tmp2)


        return x

In [37]:
tmp2

NameError: name 'tmp2' is not defined

In [38]:
#heremap_data = create_data_1(heremap_json)
traffic_map_across_city(osm,heremap_data,'JF')

1836-
[[12.96984, 77.74992], [12.97022, 77.74997], [12.97022, 77.74997], [12.9703, 77.74998], [12.9703, 77.74998], [12.9704, 77.75], [12.9704, 77.75], [12.97075, 77.75006], [12.97075, 77.75006], [12.97083, 77.75007], [12.97083, 77.75007], [12.97092, 77.75008], [12.97092, 77.75008], [12.97102, 77.75009], [12.97102, 77.75009], [12.97143, 77.75014], [12.97143, 77.75014], [12.97202, 77.75021], [12.97202, 77.75021], [12.97226, 77.75024], [12.97226, 77.75024], [12.97237, 77.75025], [12.97237, 77.75025], [12.97266, 77.75028], [12.97266, 77.75028], [12.97316, 77.75036], [12.97316, 77.75036], [12.97374, 77.75044], [12.97374, 77.75044], [12.97392, 77.75046], [12.97392, 77.75046], [12.97473, 77.75058], [12.97473, 77.75058], [12.9751, 77.75064], [12.9751, 77.75064], [12.97541, 77.75071], [12.9758, 77.75079], [12.9758, 77.75079], [12.97628, 77.75088], [12.97628, 77.75088], [12.97676, 77.75097], [12.97676, 77.75097], [12.97739, 77.75111], [12.97739, 77.75111], [12.97759, 77.75115], [12.97829, 77.751

In [32]:
tmp2

NameError: name 'tmp2' is not defined

In [20]:
a = np.array([1,2,30,-1])
a

array([ 1,  2, 30, -1])

In [21]:
np.clip(a,0,10,a)

array([ 1,  2, 10,  0])

In [22]:
for a in heremap_json['RWS'][:1]:
    print(a)


{'TY': 'TMC', 'UNITS': 'metric', 'MAP_VERSION': '201802', 'RW': [{'mid': '1139a1fd-641f-4994-b182-bda126a344e8|', 'PBT': '2018-06-25T06:04:25Z', 'FIS': [{'FI': [{'TMC': {'QD': '-', 'LE': 3.26892, 'PC': 1836, 'DE': 'College Kadugodi Road'}, 'CF': [{'TY': 'TR', 'CN': 0.72, 'SP': 26.55, 'FF': 36.0, 'SU': 26.55, 'JF': 2.63082}], 'SHP': [{'value': ['12.96984,77.74992 12.97022,77.74997 '], 'FC': 3}, {'value': ['12.97022,77.74997 12.9703,77.74998 '], 'FC': 3}, {'value': ['12.9703,77.74998 12.9704,77.75 '], 'FC': 3}, {'value': ['12.9704,77.75 12.97075,77.75006 '], 'FC': 3}, {'value': ['12.97075,77.75006 12.97083,77.75007 '], 'FC': 3}, {'value': ['12.97083,77.75007 12.97092,77.75008 '], 'FC': 3}, {'value': ['12.97092,77.75008 12.97102,77.75009 '], 'FC': 3}, {'value': ['12.97102,77.75009 12.97143,77.75014 '], 'FC': 3}, {'value': ['12.97143,77.75014 12.97202,77.75021 '], 'FC': 3}, {'value': ['12.97202,77.75021 12.97226,77.75024 '], 'FC': 3}, {'value': ['12.97226,77.75024 12.97237,77.75025 '], 'FC

In [23]:
heremap_json

{'CREATED_TIMESTAMP': '2018-06-25T06:04:49.000+0000',
 'MAP_VERSION': '',
 'RWS': [{'EBU_COUNTRY_CODE': '5',
   'EXTENDED_COUNTRY_CODE': 'F2',
   'MAP_VERSION': '201802',
   'RW': [{'DE': 'NH-207',
     'FIS': [{'FI': [{'CF': [{'CN': 0.72,
           'FF': 36.0,
           'JF': 2.63082,
           'SP': 26.55,
           'SU': 26.55,
           'TY': 'TR'}],
         'SHP': [{'FC': 3, 'value': ['12.96984,77.74992 12.97022,77.74997 ']},
          {'FC': 3, 'value': ['12.97022,77.74997 12.9703,77.74998 ']},
          {'FC': 3, 'value': ['12.9703,77.74998 12.9704,77.75 ']},
          {'FC': 3, 'value': ['12.9704,77.75 12.97075,77.75006 ']},
          {'FC': 3, 'value': ['12.97075,77.75006 12.97083,77.75007 ']},
          {'FC': 3, 'value': ['12.97083,77.75007 12.97092,77.75008 ']},
          {'FC': 3, 'value': ['12.97092,77.75008 12.97102,77.75009 ']},
          {'FC': 3, 'value': ['12.97102,77.75009 12.97143,77.75014 ']},
          {'FC': 3, 'value': ['12.97143,77.75014 12.97202,77.7502

In [24]:
heremap_data['patch'] = heremap_data['PC'].map(str) + heremap_data['QD']

In [25]:
heremap_data

,EBU_COUNTRY_CODE,EXTENDED_COUNTRY_CODE,UNITS,DE,LI,PBT,mid,LE,PC,QD,SHP,FF,CN,JF,SP,TY,patch
0,5,F2,metric,NH-207,503+01830,2018-07-07 12:01:08.420579,1139a1fd-641f-4994-b182-bda126a344e8|,3.26892,1836,-,"[{'value': ['12.96984,77.74992 12.97022,77.749...",36.0,0.72,2.63082,26.55,TR,1836-
1,5,F2,metric,NH-207,503-01830,2018-07-07 12:01:08.420579,53dd56cf-f0e0-44b6-9f0a-662541ea18c6|,3.26879,1835,+,"[{'value': ['12.99774,77.75833 12.99747,77.758...",36.0,0.77,3.52472,24.34,TR,1835+
2,5,F2,metric,White Field Road,503-02567,2018-07-07 12:01:08.420579,2e247b36-a678-4708-a632-91bedd35c431|,1.04761,2572,+,"[{'value': ['12.98295,77.76181 12.98295,77.761...",27.0,0.70,4.87573,14.99,TR,2572+
3,5,F2,metric,White Field Road,503-02567,2018-07-07 12:01:08.420579,2e247b36-a678-4708-a632-91bedd35c431|,2.09788,2571,+,"[{'value': ['12.98373,77.75242 12.98389,77.751...",35.0,0.70,8.10438,12.00,TR,2571+
4,5,F2,metric,White Field Road,503+02567,2018-07-07 12:01:08.420579,19edbe9f-5c9a-4837-8359-a49a7a972375|,2.12029,2572,-,"[{'value': ['12.98792,77.73363 12.98779,77.734...",39.0,0.71,6.87648,17.60,TR,2572-
5,5,F2,metric,White Field Road,503+02567,2018-07-07 12:01:08.420579,19edbe9f-5c9a-4837-8359-a49a7a972375|,1.05208,2573,-,"[{'value': ['12.9838,77.75254 12.98376,77.7529...",35.0,0.70,4.18765,21.99,TR,2573-


In [26]:
tmp

NameError: name 'tmp' is not defined

In [27]:
def create_data(dictionary):
        
    RWS=dictionary['RWS']
    l=[]
    for rw in RWS:
        EBU_COUNTRY_CODE=rw['EBU_COUNTRY_CODE']
        EXTENDED_COUNTRY_CODE=rw['EXTENDED_COUNTRY_CODE']
        UNITS=rw['UNITS']
        for elem in rw['RW']:
            DE=elem['DE']
            LI=elem['LI']
            PBT=elem['PBT']
            mid=elem['mid']
            FIS=elem['FIS']
            for FI in FIS:
                elem_1=FI['FI']
                for elem_2 in elem_1:
                    LE=elem_2['TMC']['LE']
                    PC=elem_2['TMC']['PC']
                    QD=elem_2['TMC']['QD']
                    SHP=elem_2['SHP']
                    CF=elem_2['CF']
                    for elem_3 in CF:
                        FF=elem_3['FF']
                        CN=elem_3['CN']                    
                        JF=elem_3['JF']
                        SP=elem_3['SP']
                        TY=elem_3['TY']
                        l.append([EBU_COUNTRY_CODE,EXTENDED_COUNTRY_CODE,UNITS,DE,LI,PBT,mid,LE,PC,QD,SHP,FF,CN,JF,SP,TY])
    clms=['EBU_COUNTRY_CODE','EXTENDED_COUNTRY_CODE','UNITS','DE','LI','PBT','mid','LE','PC','QD','SHP','FF','CN','JF','SP','TY']                    
    df=pd.DataFrame(l,columns=clms)
    df['PBT']=datetime.datetime.now()

    return df  

In [28]:
df = create_data(heremap_json)

In [29]:
df['SHP'][0]

[{'FC': 3, 'value': ['12.96984,77.74992 12.97022,77.74997 ']},
 {'FC': 3, 'value': ['12.97022,77.74997 12.9703,77.74998 ']},
 {'FC': 3, 'value': ['12.9703,77.74998 12.9704,77.75 ']},
 {'FC': 3, 'value': ['12.9704,77.75 12.97075,77.75006 ']},
 {'FC': 3, 'value': ['12.97075,77.75006 12.97083,77.75007 ']},
 {'FC': 3, 'value': ['12.97083,77.75007 12.97092,77.75008 ']},
 {'FC': 3, 'value': ['12.97092,77.75008 12.97102,77.75009 ']},
 {'FC': 3, 'value': ['12.97102,77.75009 12.97143,77.75014 ']},
 {'FC': 3, 'value': ['12.97143,77.75014 12.97202,77.75021 ']},
 {'FC': 3, 'value': ['12.97202,77.75021 12.97226,77.75024 ']},
 {'FC': 3, 'value': ['12.97226,77.75024 12.97237,77.75025 ']},
 {'FC': 3, 'value': ['12.97237,77.75025 12.97266,77.75028 ']},
 {'FC': 3, 'value': ['12.97266,77.75028 12.97316,77.75036 ']},
 {'FC': 3, 'value': ['12.97316,77.75036 12.97374,77.75044 ']},
 {'FC': 3, 'value': ['12.97374,77.75044 12.97392,77.75046 ']},
 {'FC': 3, 'value': ['12.97392,77.75046 12.97473,77.75058 ']},
 {

In [30]:
e = Xml.parse('vehicle_properties.xml').getroot()
vehicle_length = {}
for i in e:

    vehicle_length[i.attrib['id']] = float(i.attrib['length'])

In [44]:
import itertools

In [49]:
def traffic_map_across_city(df):

    data_2=df
    output = {}
    for i in (data_2.index.tolist()):
        tmp2=[]
        tmp=re.findall("\d*\.\d+", data_2['SHP'][i])
        tmp=[float(i) for i in tmp]

        it = iter(tmp)
        for z in it:
            tmp2.append([z, next(it)])

        tmp2 = list(tmp2 for tmp2,_ in itertools.groupby(tmp2))
        output[data_2['patch'][i]] = tmp2


    return output

In [50]:
traffic_map_across_city(heremap_data)

{'1835+': [[12.99774, 77.75833],
  [12.99747, 77.75838],
  [12.9972, 77.75842],
  [12.99709, 77.75844],
  [12.99698, 77.75844],
  [12.99688, 77.75844],
  [12.99668, 77.75841],
  [12.99656, 77.75839],
  [12.99637, 77.75836],
  [12.99614, 77.75833],
  [12.99595, 77.7583],
  [12.99582, 77.75827],
  [12.9957, 77.75824],
  [12.99559, 77.7582],
  [12.99548, 77.75816],
  [12.99525, 77.75808],
  [12.99499, 77.75798],
  [12.9946, 77.75782],
  [12.99451, 77.75778],
  [12.99439, 77.75773],
  [12.99431, 77.7577],
  [12.99399, 77.75761],
  [12.99373, 77.75755],
  [12.99363, 77.75753],
  [12.99346, 77.75751],
  [12.99321, 77.75747],
  [12.99311, 77.75745],
  [12.99301, 77.75741],
  [12.99292, 77.75736],
  [12.99279, 77.75726],
  [12.9913, 77.75607],
  [12.99098, 77.75582],
  [12.99014, 77.75521],
  [12.98993, 77.75507],
  [12.98966, 77.75493],
  [12.98928, 77.75476],
  [12.98864, 77.7545],
  [12.98843, 77.75441],
  [12.988, 77.75423],
  [12.98652, 77.75362],
  [12.98618, 77.75349],
  [12.98592, 77.7

In [102]:
e = Xml.parse(sumo_filename).getroot()

output = {}
for i in e:
    if i.tag == 'edge' and (i.attrib['id'] in incoming_sumo_roads):
        all_lanes_polyline = []
        for j in i:      
            if j.tag == 'lane':
                list_points = []
                for ll in j.attrib['shape'].split():
                    list_points.append(get_transform([float(i) for i in ll.split(',')]))
                    
                all_lanes_polyline.append(list_points)
                    
        output[i.attrib['id']] = np.around(np.average(all_lanes_polyline, axis = 0), 5).tolist()
        


In [103]:
output

{'122516806': [[12.98359, 77.75379],
  [12.98371, 77.75284],
  [12.98374, 77.75268],
  [12.98373, 77.75266],
  [12.98373, 77.75263]],
 '143215059': [[12.9943, 77.75762],
  [12.99376, 77.75751],
  [12.99364, 77.75749],
  [12.99324, 77.75746],
  [12.99306, 77.75737],
  [12.99291, 77.75727],
  [12.99281, 77.75719],
  [12.99192, 77.75651],
  [12.99134, 77.75605],
  [12.9909, 77.7557],
  [12.99003, 77.75508],
  [12.98963, 77.75487],
  [12.98875, 77.75452],
  [12.9884, 77.7544],
  [12.98797, 77.75426],
  [12.98711, 77.75395],
  [12.98643, 77.75367],
  [12.98584, 77.75344],
  [12.98498, 77.75309],
  [12.9841, 77.75272],
  [12.98387, 77.75262],
  [12.98383, 77.75259]],
 '28043853': [[12.95735, 77.74602],
  [12.95738, 77.74603],
  [12.95791, 77.74627],
  [12.95835, 77.74647],
  [12.95861, 77.74658],
  [12.95932, 77.74696],
  [12.96102, 77.74788],
  [12.96122, 77.74795],
  [12.96134, 77.74795],
  [12.96145, 77.74794],
  [12.9616, 77.74796],
  [12.96173, 77.74797],
  [12.96184, 77.74798],
  [12.9

In [69]:
for i, j in output.items():
    print(i, len(j))

122516806_0 5
28043853_1 56
28043853_0 56
143215059_0 22
122516806_1 5
580921886#0_0 3
143215059_1 22
580921886#0_1 3


In [70]:
with open('../hoodi_test/traffic_flow.json') as f:
    traffic_flow = json.load(f)

In [75]:
incoming_sumo_roads = []
for i in traffic_flow.values():
    for j in i:
        incoming_sumo_roads.append(j['from'])
incoming_sumo_roads = list(set(incoming_sumo_roads))

In [76]:
incoming_sumo_roads

['580921886#0', '143215059', '122516806', '28043853']

In [85]:
s = [[[12.983579297180547, 77.75379090101039],
   [12.983698402675138, 77.7528379258017],
   [12.983720009114066, 77.75268088362134],
   [12.983716498067741, 77.75266064098597],
   [12.983712266806783, 77.7526237605681]],
  [[12.983608735953588, 77.75379478315965],
   [12.983727841448179, 77.7528419928152],
   [12.983750168101738, 77.75268032902859],
   [12.983745846813951, 77.75265611181185],
   [12.983742515821284, 77.75262644109972]]]

In [86]:
d = np.array([[[12.983579297180547, 77.75379090101039],
   [12.983698402675138, 77.7528379258017],
   [12.983720009114066, 77.75268088362134],
   [12.983716498067741, 77.75266064098597],
   [12.983712266806783, 77.7526237605681]],
  [[12.983608735953588, 77.75379478315965],
   [12.983727841448179, 77.7528419928152],
   [12.983750168101738, 77.75268032902859],
   [12.983745846813951, 77.75265611181185],
   [12.983742515821284, 77.75262644109972]]])

In [87]:
d

array([[[12.9835793 , 77.7537909 ],
        [12.9836984 , 77.75283793],
        [12.98372001, 77.75268088],
        [12.9837165 , 77.75266064],
        [12.98371227, 77.75262376]],

       [[12.98360874, 77.75379478],
        [12.98372784, 77.75284199],
        [12.98375017, 77.75268033],
        [12.98374585, 77.75265611],
        [12.98374252, 77.75262644]]])

In [96]:
np.average(s, axis = 0)

array([[12.98359402, 77.75379284],
       [12.98371312, 77.75283996],
       [12.98373509, 77.75268061],
       [12.98373117, 77.75265838],
       [12.98372739, 77.7526251 ]])

In [91]:
np.mean([12.9835793, 12.98360874])

12.98359402

In [107]:
e = Xml.parse(sumo_filename).getroot()

for i in e:
    if i.tag == 'location':
        loc = [float(i) for i in i.attrib['origBoundary'].split(',')]
        points = [float(i) for i in i.attrib['convBoundary'].split(',')]
        

In [108]:
loc

[77.735568, 12.957348, 77.75767, 12.994528]

In [109]:
points

[0.0, -0.0, 2391.16, 4129.88]